# Data Collection and Analysis

# Assignment 3: Preprocessing

This assignment will take you through some of the required steps for preprocessing your own data later on. 

Here, we'll use the data we collected during the demonstration experiment (hint: open up the experiment once more to check out the durations of all the phases)

You'll to this assignment individually on you own computer (or one of the university computers). 

You are likely to run into many errors. THIS IS NORMAL. A big part of scientific computing is solving problems. Sometimes, when you enter one of your error messages into google, you will not get a solution to your problem. Ask us for help early! Asking the correct question is hard!

You can sequentially run the chunks of code in this script and they will gradually take you through the preprocessing of our demo data. Sometimes you are asked to fill in some code (it will be marked with #TODO) and sometimes you are asked to interpret output using text. 


```PSMCV-2.2024-2025.1```

Authors: 
- Robbert van der Mijn, w.r.van.der.mijn@rug.nl, 2022
- Mark Span, m.m.span@rug.nl, 2024

See also the tutorial at: https://pydatamatrix.eu/1.0/eyelinkparser/

_and:_

[A very similar explanation by Sebastiaan](https://youtu.be/PtUmhQ2vupw?t=1)

_and definitely:_

Mathôt S, Vilotijević A. Methods in cognitive pupillometry: Design, preprocessing, and statistical analysis. Behav Res Methods. 2023 Sep;55(6):3055-3077. doi: 10.3758/s13428-022-01957-7. Epub 2022 Aug 26. PMID: 36028608; PMCID: PMC10556184.

### Additional References

Kahneman, D., & Beatty, J. (1966). Pupil diameter and load on memory. Science, 154(3756), 1583–1585. https://doi.org/10.1126/science.154.3756.1583

Mathôt, S., (2018). Pupillometry: Psychology, Physiology, and Function. Journal of Cognition. 1(1), p.16. https://doi.org/10.5334/joc.18

Some of these modules are not installed by default with Python in your computer, install them first. This syntax will (try to) do so if needed. There is no need for you to understand how this works (yet..).

In [ ]:
try:
    import gdown  
except ImportError:
    print("Installing GDOWN")
    !pip install gdown

In [ ]:
try:
    import eyelinkparser  
except ImportError:
    print("Installing eyelinkparser")
    !pip install eyelinkparser

In [ ]:
try:
    import time_series_test
except ImportError:
    print("Installing time_series_test")
    !pip install time_series_test

In [ ]:
try:
    import datamatrix
except ImportError:
    print("Installing datamatrix")
    !pip install datamatrix==1.0.13 -c conda-forge

## Get Data
First we will download data that we collected before: this is data from the demo-experiment I have shown in earlier meetings...

The data will be downloaded to a directory named "data", that will be created in the current directory.

In [ ]:
import gdown
url = r'https://drive.google.com/drive/folders/1aJSK0F6vaN-scfkr1xExg3x_FRK8Gd9B'
gdown.download_folder(url)

It is common to start your analysis script with loading modules you will use later on. 

In [1]:
from datamatrix import (
    plot,
    operations as ops,
    series as srs,
    functional as fnc,
    SeriesColumn,
)
from eyelinkparser import parse, defaulttraceprocessor
import numpy as np

from matplotlib import pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager

# This is to get rid of some warnings while plotting, telling us we don't have the correct fonts installed.
matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
mpl.rcParams['font.family'] = "sans-serif"
mpl.rcParams['font.sans-serif'] = "DejaVu Sans"

The following lines define our own custom function that we can use to get our data from the folder called "data". It wraps around the "parse" function which comes with eyelinkparser and returns its result. The first time you run it, it will take some time. The memoize decorator makes sure that the result is stored on your computer, so it will be much faster the second time. 

See that we will not use the "automatic blinkreconstruction". This is for demonstration purposed: the automatic, "advanced" blinkreconstruction __is__ the preferred option.


In [2]:
@fnc.memoize(persistent=True)
def get_data():
    
    dm = parse(
        traceprocessor = defaulttraceprocessor(
          blinkreconstruct = False, 
          downsample = None
        )
    )
    return dm

#get_data.clear() # you can run this line to clear the stored output of the function
dm = get_data()

Great! We have now a data table containing the data of all participants. 

Let's inspect the information we have in our data table. Can you think of some useful statements to inspect what variables there are in a data table. hint: check out the cheat sheet on https://datamatrix.cogsci.nl/

ASSIGNMENT: 

    - Print all the column names to the console
    
    - Print out the the colums named "subject_nr" and "ptrace_fixation"


In [17]:
#TODO

Those are quite some columns! Some of these colums are not very interesting to us, so we'll get rid of them momentarily. 

Each row in this table contains the information about a single trial. In this demo, each pp did 44 trials. 

ASSIGNMENT: 

    - Confirm this by printing the number of rows in this table


In [16]:
#TODO

When you printed the whole column "subject_nr" it listed for each trial to which pp that trial belongs. 

During each trial, we've been sending messages at the start and end of phases.

ASSIGNMENT:  

    - List the phases here in the correct order


In [ ]:
#TODO

The continuous data of each phase (e.g., pupil size, gaze coordinates) are stored as SeriesColumns. For example, each cell of the column "ptrace_response" contains a collection of pupil sizes for that particular trial, during the response phase. 

Most phases are always exactly the same duration (the "problem" phase is always exactly 800ms). But, some phases have variable duration ("response" just ends when the pp presses a button). However, any SeriesColumn must always be a particular "depth" (think of our table as a "3d" table; a matrix). Its depth will always be the same depth as the longest trial, but the values will be padded with a special type of data: "nan" ("not a number").

We use a little trick to concatenate (basically "paste") the phases together. Don't worry about the details of this procedure. 


In [ ]:
def my_preprocessor():
    
    dm = parse(
        traceprocessor = defaulttraceprocessor(
          blinkreconstruct = False,
          mode = 'advanced',
          downsample = None
        )
    )
    
    # define the max depth manually: baseline, problem, fixation, response and feedback
    MAX_DEPTH = 1500 + 800 + 2500 + 2000 

    # get the current max depth, we'll be cutting it off later
    max_depth = dm.ptrace_baseline.depth + dm.ptrace_problem.depth + dm.ptrace_fixation.depth + dm.ptrace_response.depth + dm.ptrace_feedback.depth

    # Create new series column that will hold our new pupil trace
    dm.pupil = SeriesColumn(depth = max_depth)

    # For each trial, trim pupil traces based on the nan values in the time trace (there are more elegant solutions I'm sure)
    for i, row in enumerate(dm):
        ptrace_baseline = row.ptrace_baseline[~np.isnan(row.ttrace_baseline)]
        ptrace_problem  = row.ptrace_problem[~np.isnan(row.ttrace_problem)]
        ptrace_fixation = row.ptrace_fixation[~np.isnan(row.ttrace_fixation)]
        ptrace_response = row.ptrace_response[~np.isnan(row.ttrace_response)]
        ptrace_feedback = row.ptrace_feedback[~np.isnan(row.ttrace_feedback)]
        
        # Concatenate and pad so they're all the same depth
        pupil = np.concatenate((ptrace_baseline, ptrace_problem, ptrace_fixation, ptrace_response, ptrace_feedback))
        pupil = np.pad(pupil, (0, max_depth - len(pupil)), "constant", constant_values = np.nan)
        
        # Write this trial to the original dm
        dm.pupil[i] = pupil
        
    # Trim the original dm to our manual depth (the feedback phase of some of the really long trials will be cut off)
    dm.pupil.depth = MAX_DEPTH
    # and return only the data we want to analyse. 
    dm = dm[("subject_nr", "pupil", "count_trial_sequence", "correct", "difficulty", "practice", "response_time")]
    
    return dm

## Visualisation

From the matplotlib package we've imported pyplot (as plt), from which we can now use the plot function. 

In [ ]:
# Plot the very first trial (python starts counting at 0)
plt.figure() # creates a "new" figure
plt.plot(dm.pupil[0], color = "black")

# Another way to create a subset (in this case we create a new dm that holds the 3rd trial of pp 3)
subdm = (dm.subject_nr == 3) & (dm.count_trial_sequence == 2)
plt.figure()
plt.plot(subdm.pupil[0], color = "black");


ASSIGNMENT: 

    - plot two more trials with a blink (play around with subsetting)

In [ ]:
#TODO

## Blink reconstruction

When a pp blinks, data recorded during that period is missing or distorted.

We use srs.blinkreconstruct to create a new trace in which these are restored. 

For now, we'll call the column "pupil_reconstructed". In practice, you would overwrite the original one, or use a more practical (i.e., short) name.


In [ ]:
dm.pupil_reconstructed = srs.blinkreconstruct(dm.pupil, mode = "advanced")

Lets visualise how that reconstruction works.

We can plot the two traces simultaniously. We'll color the original one red ("alpha = .5" will also make it a bit "see through"). In most trials, the pp will not have blinked. See if those blinks you found in the previous assignment were interpolated correctly. 


In [ ]:
row_nr = 1
plt.figure()
plt.plot(dm.pupil_reconstructed[row_nr], color = "black")
plt.plot(dm.pupil[row_nr], color = "red", alpha = .5)
plt.title(row_nr)



srs.blinkreconstruction accepts a number of parameters that might change its functioning. 

ASSIGNMENT:

   In your own words, describe what the following do:
    
    - vt

#TODO

    - margin

#TODO

    - smooth_winlen 

#TODO

### Overwrite raw pupil data
Now we overwrite our raw pupil column with the reconstructed one

In [ ]:
dm.pupil = dm.pupil_reconstructed

## Averaging part 1

Our goal is to analyse the average pupil size. For example, to compare mean pupil sizes between two conditions. 

Do do so, we have two imporant steps ahead of us: 
    - Time Locking
    - Baselining
    
The next figures will illustrate why these are crucial

In [ ]:
# Define the colors of the conditions
CON_COLS = {"easy": "blue", "hard": "red"}
# for each pp, start a new figure
for s, subdm in ops.split(dm.subject_nr):    
    plt.figure()
    # now split up the table into two tables, the hard trials and the easy trials
    for c, cdm in ops.split(subdm.difficulty):
        # for each of those split up tables, go over each trial and plot the line
        for row in cdm:
            plt.plot(row.pupil, color = CON_COLS[c], alpha = .3)
    # Finally, for each figure, create a title and legend
    plt.title("pp {}".format(s))
    

That's quite messy!

We're expecting the red lines to be higher than blue lines. But if that difference exists, it's currently lost in intertrial variation (noise).

Let's use the plot.trace function anyway to average all these traces (we'll actually already see a difference!)


In [ ]:
CON_COLS = {"easy": "blue", "hard": "red"}
plt.figure()
for c, cdm in ops.split(dm.difficulty):
    # Notice that plot.trace is a different function than plt.plot! This one can accept a whole bunch of trials, calculated the average and sd at each time point and then plots it for us.
    plot.trace(cdm.pupil, color = CON_COLS[c], label = c, alpha = .3)
# create a legend
plt.legend(frameon = False)


### Baseline

Next step: Baselining!

We expect the difference between conditions during the baseline period (in our case, the 1500 ms before we start showing the problem) is due to noise. So we subtract the average pupil size during a short baseline window of each trial from all the other pupil values. In our case, we'll use the period between 1480 ms and 1500 ms to calculate the baseline. 

srs.baseline creates a new trace in in which exactly that is done. 


In [ ]:
dm.pupil_baselined = srs.baseline(dm.pupil, dm.pupil, bl_start = 1480, bl_end = 1500)


## Averaging part 2

Now do the exact same visualisation as earlier, but now with the baselined traces


In [ ]:
CON_COLS = {"easy": "blue", "hard": "red"}
for s, subdm in ops.split(dm.subject_nr):    
    plt.figure()
    for c, cdm in ops.split(subdm.difficulty):
        for row in cdm:
            plt.plot(row.pupil_baselined, color = CON_COLS[c], alpha = .3)
    plt.title("pp {}, baselined".format(s))

ASSIGNMENT: 

    - Explain in your own words why we see this "bow tie"-like figure
  

#TODO

And again, we can use plot.trace to visualise the means, compare this figure top the non-baselined figure!

In [ ]:
CON_COLS = {"easy": "blue", "hard": "red"}
plt.figure()
for c, cdm in ops.split(dm.difficulty):
    plot.trace(cdm.pupil_baselined, color = CON_COLS[c], label = c, alpha = .3)
plt.legend(frameon = False)

That looks quite nice! 

ASSIGNMENT: 

    - Why do you think the line becomes all jittery after about 6,5 seconds?

#TODO

### Time Lock 
(not neccesary for this demo, but could be crucial for you later!)

Baselining only works when your traces are time locked. In our demo, the problem is always shown at timepoint 1500

### Downsample
In this demo, we've recorded pupil size at 1000Hz. Each sample represents a snapshot of the pupil size for the next millisecond. 

But, changes in pupil size from 1ms to the next are unlikely to reflect a cognitive function in which we're interested. Pupil responses are much slower. Therefore, to reduce the amount of data we store, we downsample to a lower frequency. 

We could just take every 10th sample and call it a day. But that could lead to aliasing. Instead, we take every 10th sample, but replace it with the mean of the 10 surrounding samples. srs.downsample helps you with this. Here, we'll visualize the effect.


First determine the inter sample duration (verify that this is 1 ms, because we sampled at 1000Hz)

In [ ]:
SAMPDUR = dm.ttrace_baseline[1, 1] - dm.ttrace_baseline[1, 0]

print(SAMPDUR)

ASSIGNMENT: 

    - What would SAMPDUR have been if we'd set the Eyelink to sample at 250 Hz?

#TODO

Let's determine that we want to downsample by a factor of 10 (i.e., 1000Hz becomes 100Hz)

In [ ]:
DS = 10

Let's plot the first 300 samples (i.e., 300ms) of the very first trial

In [ ]:
DUR = 300
pupil = dm.pupil[0, :DUR]
time = np.arange(DUR) # will be a vector of time points (x axis)
plt.figure()
plt.plot(time, pupil, "-")
plt.xlabel("Time (ms)")
plt.ylabel("Pupil")

Nothing special so far

Now take that same segment, downsample it and plot both in one figure

In [ ]:
pupil_ds = srs.downsample(dm.pupil[0, :DUR], DS)
time_ds = np.arange((DS * SAMPDUR)/2, DUR, step = DS * SAMPDUR) # a new time vector must also be created

Using the arange function, we created a new vector of the exact same lengths as the pupil trace. The number represent the time points at which those pupil sizes were recorded. 

So we start counting at 0, up to the duration of the trace ("DUR"), with steps of the original inter sample duration multiplied with the downsample factor.

And to make this even more complicated, we don't acturally start counting at 0, but down half a sample duration further: "(DS * SAMPDUR)/2"


In [ ]:
plt.figure()
plt.plot(time, pupil, "-")
plt.plot(time_ds, pupil_ds, "-")
plt.xlabel("Time (ms)")
plt.ylabel("Pupil")

ASSIGNMENT: 

    - How many datapoints are in pupil, and in pupil_ds? hint: use pythons "len" function


In [ ]:
#TODO

Let's look a little closer

In [ ]:
ZOOM_IN = 60
plt.figure()
plt.plot(time[:ZOOM_IN], pupil[:ZOOM_IN], "-", color = "blue")
plt.plot(time[:ZOOM_IN], pupil[:ZOOM_IN], ".", color = "blue", ms = 10)
plt.plot(time_ds[:int(ZOOM_IN/DS)], pupil_ds[:int(ZOOM_IN/DS)], "-", color = "red")
plt.plot(time_ds[:int(ZOOM_IN/DS)], pupil_ds[:int(ZOOM_IN/DS)], ".", color = "red", ms = 10)
plt.xlabel("Time (ms)")
plt.ylabel("Pupil")

#### Apply downsampling

Above, we only applied downsampling to a small segment of our data. 
Now, we apply it to the whole pupil trace


In [ ]:
print("Samples per trial before downsampling: {}".format(dm.pupil.depth))
dm.pupil = srs.downsample(dm.pupil, by = 10)
print("Samples per trial after downsampling: {}".format(dm.pupil.depth))

### Reducing file size 

Before putting the whole pipeline together to run the preprocessing, we make sure to only store the data that we know we'll use later for the analysis. Therefore, go back to the assignment where you printed all the column names and determine which ones you want to keep.

ASSIGNMENT: 

    - Keep only the columns you think are neccesary for analysis

Here's an example on how to only keep these three columns.


In [ ]:
dm_small = dm[("subject_nr", "pupil", "count_trial_sequence")]

In [ ]:
#TODO


## Creating your own pipeline
You now know all the ingredients to create your own pipeline function that parses the raw data files, and does your preprocessing. 

This function looks kinda similar to the one we used earlier in this assignment. It's now your job to find all the crucial pieces and add them to our final function: "my_preprocessor()"


In [ ]:
### ANSWER
@fnc.memoize(persistent=True)
def my_preprocessor():
    
    dm = parse(
        traceprocessor = defaulttraceprocessor(
          blinkreconstruct = False, 
          downsample = None,
        )
    )
    
    # define the max depth manually: baseline, problem, fixation, response and feedback
    MAX_DEPTH = 1500 + 800 + 2500 + 2000 

    # get the current max depth, we'll be cutting it off later
    max_depth = dm.ptrace_baseline.depth + dm.ptrace_problem.depth + dm.ptrace_fixation.depth + dm.ptrace_response.depth + dm.ptrace_feedback.depth

    # Create new series column that will hold our new pupil trace
    dm.pupil = SeriesColumn(depth = max_depth)

    # For each trial, trim pupil traces based on the nan values in the time trace (there are more elegant solutions I'm sure)
    for i, row in enumerate(dm):
        ptrace_baseline = row.ptrace_baseline[~np.isnan(row.ttrace_baseline)]
        ptrace_problem = row.ptrace_problem[~np.isnan(row.ttrace_problem)]
        ptrace_fixation = row.ptrace_fixation[~np.isnan(row.ttrace_fixation)]
        ptrace_response = row.ptrace_response[~np.isnan(row.ttrace_response)]
        ptrace_feedback = row.ptrace_feedback[~np.isnan(row.ttrace_feedback)]
        
        # Concatenate and pad so they're all the same depth
        pupil = np.concatenate((ptrace_baseline, ptrace_problem, ptrace_fixation, ptrace_response, ptrace_feedback))
        pupil = np.pad(pupil, (0, max_depth - len(pupil)), "constant", constant_values = np.nan)
        
        # Write this trial to the original dm
        dm.pupil[i] = pupil
        
    # Trim the original dm to our manual depth (the feedback phase of some of the really long trials will be cut off)
    dm.pupil.depth = MAX_DEPTH
    
    dm.pupil = srs.baseline(dm.pupil, dm.pupil, bl_start = 1480, bl_end = 1500)
    
    SAMPDUR = dm.ttrace_baseline[1, 1] - dm.ttrace_baseline[1, 0]
    DS = 10
    
    dm.pupil = srs.downsample(dm.pupil, by = DS)
    
    dm.time = SeriesColumn(dm.pupil.depth)
    dm.time = np.arange((DS * SAMPDUR)/2, dm.pupil.depth * (DS/SAMPDUR), step = DS * SAMPDUR) # a new time vector must also be created

    dm = dm[("subject_nr", "pupil", "time", "count_trial_sequence", "correct", "difficulty", "practice")]
    
    return dm

my_preprocessor.clear() # you can run this line to clear the stored output of the function
dm = my_preprocessor()


ASSIGNMENT: 

    - Confirm that your preprocessor works by plotting the mean pupil size over the course of a trial

In [ ]:
#TODO


## Storing

Before we move on to analysis, we store our data. 
During analysis, we will further manipulate our data. Although we have a clear analysis plan, writing the script for your analysis will be a messy job (espcecially if you do it for the first time!).
With our preprocesses data saved, we can start messing with it but easily roll back without having to run the entire preprocessing again (which can be time consuming). Also, it allows you to share your data more readily with collaborators.


In [ ]:
from pickle import dump
from datetime import datetime
import os
# Each time we save the data, we use a filename that includes the current date and time. This way we won't accidentally overwrite files. If you prefer, you can change the filename.
now = datetime.now()
filename = "pd_{}_{}.pickle".format(os.environ['JUPYTERHUB_USER'], now.strftime("%m%d%H%M%S"))
with open(filename, 'wb') as file:
    dump(dm, file)